<a href="https://colab.research.google.com/github/Adnan525/RAG/blob/main/camel_5b_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web

In [10]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.prompts import PromptTemplate

from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
from llama_index.core.response.notebook_utils import display_response

import torch
from llama_index.llms.huggingface import HuggingFaceLLM # llm
from llama_index.embeddings.huggingface import HuggingFaceEmbedding # embedding
from llama_index.core import Settings # pass llm and embedding

from llama_index.core import VectorStoreIndex # GPTVectorStoreIndex has been renamed to VectorStoreIndex, but it is only a cosmetic change. Please see the following links for more details on usage.

In [11]:
from google.colab import userdata
hf_token = userdata.get("hf_token")

In [12]:
!mkdir -p "data/chess/"
!wget "http://dspace.vnbrims.org:13000/jspui/bitstream/123456789/4980/1/Chess_%20The%20Complete%20Guide%20To%20Chess%20-%20Master_%20Chess%20Tactics%2C%20Chess%20Openings%2C%20and%20Chess%20Strategies.pdf" -O "data/chess/chess.pdf"

--2024-05-29 01:02:30--  http://dspace.vnbrims.org:13000/jspui/bitstream/123456789/4980/1/Chess_%20The%20Complete%20Guide%20To%20Chess%20-%20Master_%20Chess%20Tactics%2C%20Chess%20Openings%2C%20and%20Chess%20Strategies.pdf
Resolving dspace.vnbrims.org (dspace.vnbrims.org)... 124.108.19.235
Connecting to dspace.vnbrims.org (dspace.vnbrims.org)|124.108.19.235|:13000... connected.
HTTP request sent, awaiting response... 200 
Length: 3003524 (2.9M) [application/pdf]
Saving to: ‘data/chess/chess.pdf’

data/chess/chess.pd 100%[===================>]   2.86M  1.62MB/s    in 1.8s    

2024-05-29 01:02:32 (1.62 MB/s) - ‘data/chess/chess.pdf’ saved [3003524/3003524]



In [27]:
!wget "http://www.cs.columbia.edu/~vovsha/coms3101/perl/acn.pdf" -O "data/chess/columbia.pdf"

--2024-05-29 01:22:25--  http://www.cs.columbia.edu/~vovsha/coms3101/perl/acn.pdf
Resolving www.cs.columbia.edu (www.cs.columbia.edu)... 128.59.11.206
Connecting to www.cs.columbia.edu (www.cs.columbia.edu)|128.59.11.206|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34630 (34K) [application/pdf]
Saving to: ‘data/chess/columbia.pdf’

data/chess/columbia 100%[===================>]  33.82K  --.-KB/s    in 0.06s   

2024-05-29 01:22:26 (565 KB/s) - ‘data/chess/columbia.pdf’ saved [34630/34630]



In [50]:
# book 3
!wget "https://www.fide.com/FIDE/handbook/LawsOfChess.pdf" -O "data/chess/LawsOfChess.pdf"

--2024-05-29 01:45:55--  https://www.fide.com/FIDE/handbook/LawsOfChess.pdf
Resolving www.fide.com (www.fide.com)... 80.241.218.178
Connecting to www.fide.com (www.fide.com)|80.241.218.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300696 (294K) [application/pdf]
Saving to: ‘data/chess/LawsOfChess.pdf’

data/chess/LawsOfCh 100%[===================>] 293.65K   481KB/s    in 0.6s    

2024-05-29 01:45:57 (481 KB/s) - ‘data/chess/LawsOfChess.pdf’ saved [300696/300696]



In [51]:
documents = SimpleDirectoryReader("data/chess").load_data()

In [52]:
books = set()
for book in documents:
  books.add(book.metadata["file_name"])

books

{'LawsOfChess.pdf', 'chess.pdf', 'columbia.pdf'}

In [14]:
model_id = "Writer/camel-5b-hf"

## Plain prompt  template

In [15]:
query_wrapper_prompt = PromptTemplate(
    "Always answer the question, even if the context isn't helpful."
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

## Using System Prompt

In [ ]:

qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

In [ ]:
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [17]:
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=model_id,
    model_name=model_id,
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit" : True}
)

config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/25.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

In [18]:
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="hkunlp/instructor-base")

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [56]:
vector_index = VectorStoreIndex.from_documents(documents)

In [58]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What are the name of the authors for all three books?")
display_response(response)
# Mary Lou Retton is an american gymnyst

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**`Final Response:`** The authors for all three books are:
1. The Complete Guide to Chess by Logan Donovan
2. Chess Tactics, Chess Openings and Chess Strategies by Mark Taiman
3. Chess by Garry Kasparov

In [59]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("how does the Knight move in chess?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**`Final Response:`** The Knight moves in an "L" shape, taking two straight steps and one step to the right or left, and can also move one step forward or backward.

In [60]:
response = query_engine.query("knowing how Knight moves in chess and algebraic notation, if a knight in on c1, is kf7 a legal move? Explain")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**`Final Response:`** Yes, the knight on c1 is a legal move. In algebraic notation, the knight moves as kf7.

In [61]:
response = query_engine.query("I am using algebraic notation for chess. If i have a Knight on b1, what are my legal moves with the Knight? please give me square position number in algebraic notation.")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**`Final Response:`** Nf3

In [62]:
response = query_engine.query("if current position of a pawn is e2, what will be its position if it takes 1 step forward?")
display_response(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**`Final Response:`** The pawn will move to f3.